In [ ]:
import pathlib     
import pandas as pd
import numpy as np 
import tqdm        
import torch       
import gzip
import os                                             
import json                                           
import time                                           
import gzip                                           
import pprint                                         
import argparse                                       
                                                      
import numpy as np                                    
import torch                                          
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch import optim                               
import torch.nn as nn                                 
import torch.nn.functional as F                       
import sklearn                                        
from sklearn.utils import resample

import torch                          
from torch.nn.utils import weight_norm


def CRPSLoss(y_pred, y):                                                        
    return torch.sum((y_pred.cumsum(-1) - y.cumsum(-1))**2) / (199 * y_pred.shape[0])
                                                                                
YARDS_CLIP = [-15, 50]

device = 'cpu'

grad_clip = 0.25
lr_max = 0.001
lr = 0.001
eta_min = 1e-7

max_epochs = 40
n_bagging = 4

In [ ]:
# FROM https://github.com/dkumazaw/onecyclelr
# License: https://github.com/dkumazaw/onecyclelr/blob/master/LICENSE
from torch.optim import Optimizer


class OneCycleLR:
    """ Sets the learing rate of each parameter group by the one cycle learning rate policy
    proposed in https://arxiv.org/pdf/1708.07120.pdf. 

    It is recommended that you set the max_lr to be the learning rate that achieves 
    the lowest loss in the learning rate range test, and set min_lr to be 1/10 th of max_lr.

    So, the learning rate changes like min_lr -> max_lr -> min_lr -> final_lr, 
    where final_lr = min_lr * reduce_factor.

    Note: Currently only supports one parameter group.

    Args:
        optimizer:             (Optimizer) against which we apply this scheduler
        num_steps:             (int) of total number of steps/iterations
        lr_range:              (tuple) of min and max values of learning rate
        momentum_range:        (tuple) of min and max values of momentum
        annihilation_frac:     (float), fracion of steps to annihilate the learning rate
        reduce_factor:         (float), denotes the factor by which we annihilate the learning rate at the end
        last_step:             (int), denotes the last step. Set to -1 to start training from the beginning

    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> scheduler = OneCycleLR(optimizer, num_steps=num_steps, lr_range=(0.1, 1.))
        >>> for epoch in range(epochs):
        >>>     for step in train_dataloader:
        >>>         train(...)
        >>>         scheduler.step()

    Useful resources:
        https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6
        https://medium.com/vitalify-asia/whats-up-with-deep-learning-optimizers-since-adam-5c1d862b9db0
    """

    def __init__(self,
                 optimizer: Optimizer,
                 num_steps: int,
                 lr_range: tuple = (0.1, 1.),
                 momentum_range: tuple = (0.85, 0.95),
                 annihilation_frac: float = 0.1,
                 reduce_factor: float = 0.01,
                 last_step: int = -1):
        # Sanity check
        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(type(optimizer).__name__))
        self.optimizer = optimizer

        self.num_steps = num_steps

        self.min_lr, self.max_lr = lr_range[0], lr_range[1]
        assert self.min_lr < self.max_lr, \
            "Argument lr_range must be (min_lr, max_lr), where min_lr < max_lr"

        self.min_momentum, self.max_momentum = momentum_range[0], momentum_range[1]
        assert self.min_momentum < self.max_momentum, \
            "Argument momentum_range must be (min_momentum, max_momentum), where min_momentum < max_momentum"

        self.num_cycle_steps = int(num_steps * (1. - annihilation_frac))  # Total number of steps in the cycle
        self.final_lr = self.min_lr * reduce_factor

        self.last_step = last_step

        if self.last_step == -1:
            self.step()

    def state_dict(self):
        """Returns the state of the scheduler as a :class:`dict`.
        It contains an entry for every variable in self.__dict__ which
        is not the optimizer. (Borrowed from _LRScheduler class in torch.optim.lr_scheduler.py)
        """
        return {key: value for key, value in self.__dict__.items() if key != 'optimizer'}

    def load_state_dict(self, state_dict):
        """Loads the schedulers state. (Borrowed from _LRScheduler class in torch.optim.lr_scheduler.py)
        Arguments:
            state_dict (dict): scheduler state. Should be an object returned
                from a call to :meth:`state_dict`.
        """
        self.__dict__.update(state_dict)

    def get_lr(self):
        return self.optimizer.param_groups[0]['lr']

    def get_momentum(self):
        return self.optimizer.param_groups[0]['momentum']

    def step(self):
        """Conducts one step of learning rate and momentum update
        """
        current_step = self.last_step + 1
        self.last_step = current_step

        if current_step <= self.num_cycle_steps // 2:
            # Scale up phase
            scale = current_step / (self.num_cycle_steps // 2)
            lr = self.min_lr + (self.max_lr - self.min_lr) * scale
            momentum = self.max_momentum - (self.max_momentum - self.min_momentum) * scale
        elif current_step <= self.num_cycle_steps:
            # Scale down phase
            scale = (current_step - self.num_cycle_steps // 2) / (self.num_cycle_steps - self.num_cycle_steps // 2)
            lr = self.max_lr - (self.max_lr - self.min_lr) * scale
            momentum = self.min_momentum + (self.max_momentum - self.min_momentum) * scale
        elif current_step <= self.num_steps:
            # Annihilation phase: only change lr
            scale = (current_step - self.num_cycle_steps) / (self.num_steps - self.num_cycle_steps)
            lr = self.min_lr - (self.min_lr - self.final_lr) * scale
            momentum = None
        else:
            # Exceeded given num_steps: do nothing
            return

        self.optimizer.param_groups[0]['lr'] = lr
        if momentum:
            self.optimizer.param_groups[0]['momentum'] = momentum

In [ ]:
def preprocess(df, predicting=False):                                                                                                                                      
    # fix team abbr                                                             
    abbr_corrections = {'BLT': 'BAL', 'CLV': 'CLE', 'ARZ': 'ARI', 'HST': 'HOU'} 
    for k, v in abbr_corrections.items():                                       
        df.loc[df.PossessionTeam == k, 'PossessionTeam'] = v                    
        df.loc[df.FieldPosition == k, 'FieldPosition'] = v                      
        df.loc[df.HomeTeamAbbr == k, 'HomeTeamAbbr'] = v                        
                                                                                
    # find offender/defender and rusher                                         
    df['Offender'] = (df.HomeTeamAbbr == df.PossessionTeam) & (df.Team == 'home') | (df.HomeTeamAbbr != df.PossessionTeam) & (df.Team == 'away')
    df['Rusher'] = df.NflIdRusher == df.NflId                                   
                                                                                
    # some NA in Dir; should be okay as speed = 0                               
    assert df[df.Dir.isna() & (df.S != 0.0)].size == 0                          
    df.Dir.fillna(0., inplace=True)                                             
                                                                                
    df['dir'] = -(df.Dir * np.pi/180. - np.pi/2.) # adjusted & radian           
    df['Y_aug'] = 53.33 - df['Y'] # y coordinates flipe                         
                                                                                
    # field postion NA when YardLine = 50                                       
    assert df.loc[df.FieldPosition.isna() & (df.YardLine != 50)].shape[0] == 0  
                                                                                
    # adjust yardline to x-axis                                                 
    __mask = ~df.FieldPosition.isna() & ((df.FieldPosition != df.PossessionTeam) & (df.PlayDirection == 'right') | (df.FieldPosition == df.PossessionTeam) & (df.PlayDirection == 'left'))
    df.loc[__mask, 'YardLine'] = 100 - df.loc[__mask, 'YardLine']               
                                                                                
    # fix Speed column for 2017 season                                          
    __2017_season = df.Season == 2017                                           
    df.loc[__2017_season, 'S'] = 10 * df.loc[__2017_season, 'Dis']              
                                                                                
    df['dx'] = df.S * np.cos(df.dir)                                            
    df['dy'] = df.S * np.sin(df.dir)                                            
    df['dy_aug'] = -df['dy']                                                    
                                                                                
    # make it always from left to right                                         
    __mask = (df.PlayDirection == 'left')                                       
    df.loc[__mask, 'X'] = 120 - df.loc[__mask, 'X'] # range 0 ~ 120             
    df.loc[__mask, 'Y'] = 53.33 - df.loc[__mask, 'Y']                           
    df.loc[__mask, 'Y_aug'] = 53.33 - df.loc[__mask, 'Y_aug']                   
    df.loc[__mask, 'dx'] = -df.loc[__mask, 'dx']                                
    df.loc[__mask, 'dy'] = -df.loc[__mask, 'dy']                                
    df.loc[__mask, 'dy_aug'] = -df.loc[__mask, 'dy_aug']                        
    df.loc[__mask, 'YardLine'] = 100 - df.loc[__mask, 'YardLine']               
                                                                                
    # create augmented feature for all rows and select during dfing             
    play_group = df.groupby('PlayId', sort=True)
    
    play_df_cols = (['Yards'] if not predicting else []) + ['YardLine', 'Season', 'GameId']
    play_df = play_group[play_df_cols].first().reset_index()
    if not predicting:
        play_df['YardsClipped'] = play_df.Yards.clip(YARDS_CLIP[0], YARDS_CLIP[1])  
                                                                                
    cols = ['X', 'Y', 'dx', 'dy', 'X', 'Y_aug', 'dx', 'dy_aug']                 
                                                                                
    features = []                                                               
    features_aug = []                                                           
    for _id, g in tqdm.tqdm(play_group):                                        
        diffense = g.loc[~g.Offender, cols].values                              
        offense = g.loc[g.Offender & ~g.Rusher, cols].values                    
        rusher = g.loc[g.Rusher, cols].values                                   
                                                                                
        f12 = diffense[:,None] - offense[None]                                  
        f34 = np.repeat((diffense - rusher)[:,None], repeats=10, axis=1)        
        f5 = np.repeat(diffense[:,[2,3,6,7]][:,None], repeats=10, axis=1)       
                                                                                
        f     = np.concatenate([f12[...,:4], f34[...,:4], f5[...,:2]], axis=-1) 
        f_aug = np.concatenate([f12[...,4:], f34[...,4:], f5[...,2:]], axis=-1) 
                                                                                
        features.append(f)                                                      
        features_aug.append(f_aug)                                              
                                                                                
    YARD_GRID = np.arange(-99, 100)[None]                                       
    EYE = np.eye(YARDS_CLIP[1]-YARDS_CLIP[0]+1)                                 
                                                                                
    features = np.stack(features).transpose((0, 3, 1, 2)) # channel first       
    features_aug = np.stack(features_aug).transpose((0, 3, 1, 2)) # channel first
                                                                                
    assert np.isnan(features).sum() == 0, f"nan found in features"              
    assert np.isnan(features_aug).sum() == 0, f"nan found in features"          
                                                                                
    play_ids = play_df.PlayId.values
    if not predicting:
        yards = EYE[play_df.YardsClipped.values + -YARDS_CLIP[0]]
        idxs_2017 = play_df.loc[play_df.Season == 2017].index.tolist()
    yard_lines = play_df.YardLine.values
    yard_mask = ((YARD_GRID <= (100 - yard_lines[:,None])) & (YARD_GRID >= -yard_lines[:,None])).astype(np.int)
    game_ids = play_df.GameId.values                                            
                  
    
    if not predicting:
        D = [features, features_aug, yards, yard_mask, game_ids]
    else:
        D = [features, features_aug, yard_mask, game_ids]
    assert all(D[0].shape[0] == d.shape[0] for d in D)
    if not predicting:
        D += [idxs_2017]                                                            
                                                                                
    return D



def run_epoch(model, loader, opt=None, scheduler=None, _epoch=-1):              
    model.eval() if opt is None else model.train()                              
    dev = next(model.parameters()).device                                       
    batch_id = 0                                                                
    total_loss = 0 # loss for log interval                                      
    epoch_loss = 0                                                              
                                                                                
    with torch.enable_grad() if opt else torch.no_grad():                       
        for _batch in loader:                                                   
            X, y, mask = [t.to(dev) for t in _batch]                            
            y_pred = model(X)                                                   
            y_pred = F.softmax(y_pred, dim=-1)                                  
            loss = CRPSLoss(y_pred, y)                                          
            batch_loss = loss.detach().to('cpu').numpy()                        
            epoch_loss += batch_loss                                            
            total_loss += batch_loss                                            
                                                                                
            if opt:                                                             
                opt.zero_grad()                                                 
                loss.backward()                                                 
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                opt.step()
                scheduler.step()
    return epoch_loss / len(loader)                                             
                                                                                
def setup_train(max_steps):                                                     
    model = NFLRushNet()                                                        
    n_all_param = sum([p.nelement() for p in model.parameters() if p.requires_grad])
    print(f'#params = {n_all_param/1e6:.2f}M')                                
                                                                                
    # initilize optimizer and learning rate scheduler                           
    optimizer = optim.Adam(model.parameters(), lr=lr)
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, max_steps, eta_min=eta_min)
    scheduler = OneCycleLR(optimizer, lr_range=(0.0005, 0.001), num_steps=max_steps)
                                                                                
    model = model.to(device)                                           
                                                                                
    return model, optimizer, scheduler                                     
                                                                                
def run_final(X, X_aug, y, mask, n_bagging=4, bagging_p=0.9, aug=True, aug_p=0.5, batch_size=64):
    ixs_tr = list(range(X.shape[0]))                                            
    n_samples = int(len(ixs_tr) * bagging_p)                                    
               
    models = []
    random_state = 1000                                                         
    for i in range(n_bagging):                                                  
        ixs = resample(ixs_tr, n_samples=n_samples, random_state=random_state)  
        train_dataset = RushDataset(X[ixs], X_aug[ixs], y[ixs], mask[ixs], aug=aug, aug_p=0.5)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        model, loss = train_loop(train_loader, bagging_ix=i)                           
        print("train loss=", loss)                                              
        random_state += 1
        models.append(model)
    return models    
        

def train_loop(train_loader, fold_ix=0, bagging_ix=0, meta={}):
                                                                                
    max_steps = max_epochs * len(train_loader)                             
                                                                                
    print(f'epoch step size = {len(train_loader)}')                           
    print(f'max steps = {max_steps}')                                         
                                                                                
    model, opt, scheduler = setup_train(max_steps)                              
                                                                                                                                                                                                                   
    for epoch_i in range(max_epochs):                       
        start = time.time()                                                     
        train_loss = run_epoch(model, train_loader, opt, scheduler, _epoch=epoch_i)
        end = time.time()                                                   
        print(f"{bagging_ix}/{epoch_i:2d}|{end-start:.2f}s|lr({opt.param_groups[0]['lr']:.7f})|loss({train_loss:.5f})")
                                                                                                                                                                                                               
    return model, train_loss

class NFLRushNet(nn.Module):                                   
    def __init__(self, dropout=0.3):                           
        super().__init__()                                     
        target_size = YARDS_CLIP[1] - YARDS_CLIP[0] + 1        
        n_channels = 10                                        
        h, w = 11, 10 # (offense, defense)                     
        lambda_ = 0.7                                          
                                                               
        self.net1 = nn.ModuleList([                            
            nn.Sequential(                                     
                nn.Conv2d(n_channels, 128, kernel_size=1),     
                nn.ReLU()                                      
            ),                                                 
            nn.Sequential(                                     
                nn.Conv2d(128, 160, kernel_size=1),            
                nn.ReLU()                                      
            ),                                                 
            nn.Sequential(                                     
                nn.Conv2d(160, 128, kernel_size=1),            
                nn.ReLU()                                      
            )                                                  
        ])                                                     
                                                               
        # pool over offense                                    
        self.max_pool1 = nn.MaxPool2d((h, 1))                  
        self.avg_pool1 = nn.AvgPool2d((h, 1))                  
                                                               
        self.bn1 = nn.BatchNorm1d(128)                         
                                                               
        self.net2 = nn.ModuleList([                            
            nn.Sequential(                                     
                nn.Conv1d(128, 160, kernel_size=1, bias=False),
                nn.ReLU(),                                     
                nn.BatchNorm1d(160)                            
            ),                                                 
            nn.Sequential(                                     
                nn.Conv1d(160, 96, kernel_size=1, bias=False), 
                nn.ReLU(),                                     
                nn.BatchNorm1d(96)                             
            ),                                                 
            nn.Sequential(                                     
                nn.Conv1d(96, 96, kernel_size=1, bias=False),  
                nn.ReLU(),                                     
                nn.BatchNorm1d(96)                             
            )                                                  
        ])                                                     
                                                               
        self.max_pool2 = nn.MaxPool1d(w)                       
        self.avg_pool2 = nn.AvgPool1d(w)                       
                                                               
                                                               
        self.net3 = nn.ModuleList([                            
            nn.Sequential(                                     
                nn.Linear(96, 96, bias=False),                 
                nn.ReLU(),                                     
                nn.BatchNorm1d(96)                             
            ),                                                 
            nn.Sequential(                                     
                nn.Linear(96, 256, bias=False),                
                nn.ReLU(),                                     
                nn.LayerNorm(256)                              
            ),                                                 
        ])                                                     
                                                               
        self.do = nn.Dropout(dropout)                          
        self.out = nn.Linear(256, target_size)                 
                                                               
        self.apply(self.weights_init)
        
    def forward(self, inp):                                                 

        x = inp                                                             
        for i in range(len(self.net1)):                                     
            x = self.net1[i](x)                                             

        
        x = (0.3*self.max_pool1(x) + 0.7*self.avg_pool1(x)).squeeze(2)       

        x = self.bn1(x)                                                     

        for i in range(len(self.net2)):                                     
            x = self.net2[i](x)                                             

        x = (0.3*self.max_pool2(x) + 0.7*self.avg_pool2(x)).squeeze(-1)       

        for i in range(len(self.net3)):                                     
            x = self.net3[i](x)                                             

        x = self.do(x)                                                      

        x = self.out(x)                                                     

        return x                                                            
        #return F.softmax(x, dim=-1)                                        

    def wnorm(self):                                                        
        self.conv = weight_norm(self.conv, name="weight")                   
        for i in range(len(self.pre)):                                      
            self.pre[i] = weight_norm(self.pre[i], name="weight")           

    @staticmethod                                                           
    def init_weight(weight):                                                
        nn.init.uniform_(weight, -0.1, 0.1)                                 

    @staticmethod                                                           
    def init_bias(bias):                                                    
        nn.init.constant_(bias, 0.0)                                        

    @staticmethod                                                           
    def weights_init(m):                                                    
        classname = m.__class__.__name__                                    
        if classname.find('Linear') != -1 or classname.find('Conv2d') != -1:
            if hasattr(m, 'weight') and m.weight is not None:               
                NFLRushNet.init_weight(m.weight)                            
                if hasattr(m, 'bias') and m.bias is not None:               
                    NFLRushNet.init_bias(m.bias)
                    
class RushDataset(Dataset):                                                   
   def __init__(self, X, X_aug, y, mask, aug=True, aug_p=0.5, tta=False):    
       D = [X, X_aug, y, mask]                                               
       assert all(d.shape[0] == D[0].shape[0] for d in D)                    
                                                                             
       self.X = [torch.Tensor(X_aug), torch.Tensor(X)]                       
       self.y = torch.Tensor(y)                                              
       self.mask = torch.Tensor(mask)                                        
       self.aug = aug                                                        
       self.aug_p = aug_p                                                    
       self.tta = tta                                                        
                                                                             
   def __getitem__(self, idx):                                               
       if not self.tta:                                                 
           xix = int(self.aug and np.random.binomial(1, self.aug_p))         
           return self.X[xix][idx], self.y[idx], self.mask[idx]              
       else:                                                                 
           return self.X[0][idx], self.X[1][idx], self.y[idx], self.mask[idx]
                                                                             
   def __len__(self):                                                        
       return self.X[0].shape[0] 


In [ ]:
train_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)
D = preprocess(train_df)

In [ ]:
X, X_aug, y, mask, groups, idx_2017 = D
models = run_final(X, X_aug, y, mask, batch_size=64, n_bagging=n_bagging)

In [ ]:
from kaggle.competitions import nflrush
import pandas as pd

# You can only call make_env() once, so don't lose it!
env = nflrush.make_env()
iter_test = env.iter_test()

def pred(test_df, sample_prediction_df):
    D_test = preprocess(test_df, predicting=True)
    X, X_aug, *_ = D_test
    X = torch.Tensor(X)
    X_aug = torch.Tensor(X_aug)
    preds = []
    for m in models:
        m.eval()
        with torch.no_grad():
            preds.append(m(X) * 0.5 + m(X_aug) * 0.5)
            #preds.append(F.softmax(m(X), dim=-1) * 0.5 + F.softmax(m(X_aug), dim=-1) * 0.5)
    preds = torch.cat(preds, dim=0).mean(dim=0)
    preds = F.softmax(preds, dim=-1).cumsum(dim=-1)
    sample_prediction_df.iloc[0] = 0.
    sample_prediction_df.iloc[0, 84:150] = preds.numpy()
    sample_prediction_df.iloc[0, 150:] = 1.
    sample_prediction_df.clip(0., 1., inplace=True)

In [ ]:
for test_df, sample_prediction_df in iter_test:
    pred(test_df, sample_prediction_df)
    env.predict(sample_prediction_df)

In [ ]:
env.write_submission_file()

In [ ]:
# We've got a submission file!
import os
print([filename for filename in os.listdir('/kaggle/working') if '.csv' in filename])